In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('nba.db')
conn.execute("""ATTACH 'covid19.db' AS covid19""")
conn.execute("""ATTACH 'twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH 'imdb.db' AS imdb""")

## 50. 從 twElection2020 資料庫查詢三組總統候選人在各縣市的得票數，參考下列的預期查詢結果。

In [3]:
summarize_presidential_votes_from_twelection2020 =\
"""
-- SQL 查詢語法起點

SELECT soong_yu.county,
       soong_yu.soong_yu_votes,
       han_chang.han_chang_votes,
       tsai_lai.tsai_lai_votes
  FROM (SELECT admin_regions.county,
               SUM(votes) AS soong_yu_votes
          FROM presidential
          JOIN candidates
            ON presidential.candidate_id = candidates.id
          JOIN admin_regions
            ON presidential.admin_region_id = admin_regions.id
         WHERE candidates.id = 1
         GROUP BY admin_regions.county) soong_yu
  JOIN (SELECT admin_regions.county,
               SUM(votes) AS han_chang_votes
          FROM presidential
          JOIN candidates
            ON presidential.candidate_id = candidates.id
          JOIN admin_regions
            ON presidential.admin_region_id = admin_regions.id
         WHERE candidates.id = 2
         GROUP BY admin_regions.county) han_chang
    ON soong_yu.county = han_chang.county
  JOIN (SELECT admin_regions.county,
               SUM(votes) AS tsai_lai_votes
          FROM presidential
          JOIN candidates
            ON presidential.candidate_id = candidates.id
          JOIN admin_regions
            ON presidential.admin_region_id = admin_regions.id
         WHERE candidates.id = 3
         GROUP BY admin_regions.county) tsai_lai
    ON soong_yu.county = tsai_lai.county;

-- SQL 查詢語法終點
"""

presidential_votes_from_twelection2020 = pd.read_sql(summarize_presidential_votes_from_twelection2020, conn)
presidential_votes_from_twelection2020

,county,soong_yu_votes,han_chang_votes,tsai_lai_votes
0,南投縣,13315,133791,152046
1,嘉義市,6204,56269,99265
2,嘉義縣,11138,98810,197342
3,基隆市,11878,99360,114966
4,宜蘭縣,10739,90010,173657
5,屏東縣,14021,179353,317676
6,彰化縣,35060,291835,436336
7,新北市,112620,959631,1393936
8,新竹市,14103,102725,144274
9,新竹縣,18435,154224,152380


## 51. 從 covid19 資料庫查詢截至 2021-03-31 美國前十大確診人數的州別

In [4]:
summarize_confirmed_by_us_states_from_covid19 =\
"""
-- SQL 查詢語法起點

SELECT lookup_table.Province_State,
       SUM(daily_report.Confirmed) AS Confirmed
  FROM daily_report
  JOIN lookup_table
    ON daily_report.Combined_Key = lookup_table.Combined_Key
 WHERE lookup_table.Country_Region = 'US'
 GROUP BY lookup_table.Province_State
 ORDER BY Confirmed DESC
 LIMIT 10;

-- SQL 查詢語法終點
"""

confirmed_by_us_states_from_covid19 = pd.read_sql(summarize_confirmed_by_us_states_from_covid19, conn)
confirmed_by_us_states_from_covid19

,Province_State,Confirmed
0,California,3668277
1,Texas,2791910
2,Florida,2057735
3,New York,1873138
4,Illinois,1244499
5,Georgia,1059548
6,Pennsylvania,1027678
7,Ohio,1017566
8,North Carolina,914132
9,New Jersey,908816


## 52. 從 covid19 資料庫查詢截至 2021-03-31 台灣、日本、中國、南韓與新加坡五個國家的確診與死亡人數的資訊


In [5]:
# 記住：這種算各國的某個敘述統計值的時候，一定要先做到GROUP BY
summarize_confirmed_deaths_of_east_asia_from_covid19 =\
"""
-- SQL 查詢語法起點

SELECT lookup_table.Country_Region,
       SUM(daily_report.Confirmed) AS Confirmed,
       SUM(daily_report.Deaths) AS Deaths
  FROM daily_report
  JOIN lookup_table
    ON daily_report.Combined_Key = lookup_table.Combined_Key
 WHERE lookup_table.Country_Region IN ('China', 'Japan', 'Korea, South', 'Singapore', 'Taiwan')
 GROUP BY lookup_table.Country_Region;

-- SQL 查詢語法終點
"""

confirmed_deaths_of_east_asia_from_covid19 = pd.read_sql(summarize_confirmed_deaths_of_east_asia_from_covid19, conn)
confirmed_deaths_of_east_asia_from_covid19

,Country_Region,Confirmed,Deaths
0,China,101732,4841
1,Japan,474641,9155
2,"Korea, South",103639,1735
3,Singapore,60381,30
4,Taiwan,1030,10


## 53. 從 imdb 資料庫查詢出現最多次的導演為誰

In [6]:
find_most_frequent_directors_from_imdb =\
"""
-- SQL 查詢語法起點

SELECT director,
       counts
  FROM (SELECT director,
               COUNT(*) AS counts
          FROM movies
         GROUP BY director) AS counts_by_directors
 WHERE counts = (SELECT MAX(counts)
  FROM (SELECT director,
               COUNT(*) AS counts
          FROM movies
         GROUP BY director) AS counts_by_directors);

-- SQL 查詢語法終點
"""
most_frequent_directors_from_imdb = pd.read_sql(find_most_frequent_directors_from_imdb, conn)
most_frequent_directors_from_imdb

,director,counts
0,Christopher Nolan,7
1,Martin Scorsese,7
2,Stanley Kubrick,7


## 54. 從 imdb 資料庫查詢出現最多次的演員為誰

In [7]:
find_most_frequent_actors_from_imdb =\
"""
-- SQL 查詢語法起點

SELECT actor_id,
       actors.name,
       counts
  FROM (SELECT actor_id,
               COUNT(*) AS counts
          FROM casting
         GROUP BY actor_id) counts_by_actors
  JOIN actors
    ON counts_by_actors.actor_id = actors.id
 WHERE counts = (SELECT MAX(counts)
                   FROM (SELECT actor_id,
                                COUNT(*) AS counts
                           FROM casting
                          GROUP BY actor_id) counts_by_actors);
       

-- SQL 查詢語法終點
"""
most_frequent_actors_from_imdb = pd.read_sql(find_most_frequent_actors_from_imdb, conn)
most_frequent_actors_from_imdb

,actor_id,name,counts
0,2472,Robert De Niro,9


## 55. 從 imdb 資料庫查詢評等大於等於 8.8（rating >= 8.8）電影的導演以及第一主角（ord = 1）

In [8]:
find_lead_actors_from_imdb =\
"""
-- SQL 查詢語法起點

SELECT movies.title,
       movies.director,
       actors.name AS lead_actor
  FROM movies
  JOIN casting
    ON casting.movie_id = movies.id
  JOIN actors
    ON casting.actor_id = actors.id
 WHERE movies.rating >= 8.8 AND casting.ord = 1;

-- SQL 查詢語法終點
"""

lead_actors_from_imdb = pd.read_sql(find_lead_actors_from_imdb, conn)
lead_actors_from_imdb

,title,director,lead_actor
0,The Shawshank Redemption,Frank Darabont,Tim Robbins
1,The Godfather,Francis Ford Coppola,Marlon Brando
2,The Godfather: Part II,Francis Ford Coppola,Al Pacino
3,The Dark Knight,Christopher Nolan,Christian Bale
4,12 Angry Men,Sidney Lumet,Martin Balsam
5,Schindler's List,Steven Spielberg,Liam Neeson
6,The Lord of the Rings: The Return of the King,Peter Jackson,Noel Appleby
7,Pulp Fiction,Quentin Tarantino,Tim Roth
8,"The Good, the Bad and the Ugly",Sergio Leone,Eli Wallach
9,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,Alan Howard
